In [45]:
#Importing the required libraries
import requests
from datetime import datetime, timedelta
import json

## Sales Order

In [17]:
#Function to call the API
def api_callSO(tranDate):
    response = requests.get(f'http://127.0.0.1:8000/soData?paramDate={tranDate}')
    data = response.json()        
    tupleToInsert = tuple(data.values())
    return tupleToInsert

#Function to create a SQL insert statement
def convert_to_sqlSO(data):
    sql_statements = []
    for record in data:
        values = ', '.join([f"'{str(value)}'" for value in record])
        sql_statement = f"""INSERT INTO salesorder
    (idCustomer, idItem, createdDate, dueDate, shipDate, qty, qtyFullfilled, qtyShipped, soStatus) 
    VALUES ({values});"""
        sql_statements.append(sql_statement)
    return sql_statements

#Function to store SQL script into a file
def store_sql_scriptSO(sql_statements, file_name='so_insert.sql'):
    with open(file_name, 'w') as file:
        for statement in sql_statements:
            file.write(statement  + '\n')

In [18]:
#Starting date from the extraction
tranDate = datetime.datetime(2024, 1, 1).strftime("%Y-%m-%d")

#List that will contain the results
resultsSO = []

In [19]:
i = 0

#creting a iteration to retrieve the data and storing in into a dictionary
while True:
    #if its the first iter the starting date will be used
    if i == 0:
        resultsSO.append(api_callSO(tranDate))
    #for the following iter the last date will be used
    else:
        #this is used to get the last date and have only the date from it
        call = api_callSO(resultsSO[-1][2][:10])
        #only the year 2024 will be retrived
        if "2025" in call[2][:10]:
            break
        else:
            resultsSO.append(call)
    i += 1
    #security measuere
    if i == 10000:
        break

In [20]:
#creating the inserts statements
sql_statements_SO = convert_to_sqlSO(resultsSO)

In [21]:
#saving the statements
store_sql_scriptSO(sql_statements_SO)

## WO

In [30]:
#Function to call the API
def api_callWO(idSO, idItem, fromDate, qtyFullfilled, soStatus):
    response = requests.get(f'http://127.0.0.1:8000/woData?idSO={idSO}&idItem={idItem}&fromDate={fromDate}&qtyFullfilled={qtyFullfilled}&soStatus={soStatus}')
    data = response.json()        
    tupleToInsert = tuple(data.values())
    return tupleToInsert

#Function to create a SQL insert statement
def convert_to_sqlWO(data):
    sql_statements = []
    for record in data:
        values = ', '.join([f"'{str(value)}'" for value in record])
        sql_statement = f"""INSERT INTO workOrder 
    (idSO, idItem, createdDate, closedDate, qtyCreated, scrapQty) 
    VALUES ({values});"""
        sql_statements.append(sql_statement)
    return sql_statements

#Function to store SQL script into a file
def store_sql_scriptWO(sql_statements, file_name='woInsert.sql'):
    with open(file_name, 'w') as file:
        for statement in sql_statements:
            file.write(statement  + '\n')

In [25]:
#List that will contain the results
resultsWO = []

In [31]:
#Iterating through the sales order to retrieve the working orders data
for i in range(len(resultsSO)):
    #because the id in the SO table is a Serial and we are inserting the data on the creation of the container
    #the position of the list will be used as the id because they are the same
    resultsWO.append(api_callWO(i+1, resultsSO[i][1], resultsSO[i][2][:10], resultsSO[i][6], resultsSO[i][8]))

In [32]:
#creating the inserts statements
sql_statements_WO = convert_to_sqlWO(resultsWO)

In [33]:
#saving the statements
store_sql_scriptWO(sql_statements_WO)

## Quota

In [34]:
#Function to call the API
def api_callQuota(fromDate, idItem):
    response = requests.get(f'http://127.0.0.1:8000/quota?fromDate={fromDate}&idItem={idItem}')
    data = response.json()        
    tupleToInsert = tuple(data.values())
    return tupleToInsert

#Function to create a SQL insert statement
def convert_to_sqlQuota(data):
    sql_statements = []
    for record in data:
        values = ', '.join([f"'{str(value)}'" for value in record])
        sql_statement = f"""INSERT INTO quota 
    (period, idItem, quota) 
    VALUES ({values});"""
        sql_statements.append(sql_statement)
    return sql_statements

#Function to store SQL script into a file
def store_sql_scriptQuota(sql_statements, file_name='quotaInsert.sql'):
    with open(file_name, 'w') as file:
        for statement in sql_statements:
            file.write(statement  + '\n')

In [40]:
#List that will contain the results
resultsQuota = []

#List containing the periods for the Quota that needs to be generated
periods = ["2024-01-01",
           "2024-02-01",
           "2024-03-01",
           "2024-04-01",
           "2024-05-01",
           "2024-06-01",
           "2024-07-01",
           "2024-08-01",
           "2024-09-01",
           "2024-10-01",
           "2024-11-01",
           "2024-12-01"]

#List cointaing the ID of the items
items = ["1","2","3"]

In [41]:
for period in periods:
    for item in items:
        resultsQuota.append(api_callQuota(period, item))

In [42]:
#creating the inserts statements
sql_statements_Quota = convert_to_sqlQuota(resultsQuota)

In [43]:
#saving the statements
store_sql_scriptQuota(sql_statements_Quota)

## Sensor Data

In [49]:
#funtion to generate a list with the days beetween 2 dates
def generate_date_list(start_date, end_date):
    date_list = []
    current_date = start_date
    
    while current_date <= end_date:
        date_list.append(current_date)
        current_date += timedelta(days=1)
    
    return date_list

#Function to create a SQL insert statement
def convert_to_sqlSensor(data):
    sql_statements = []
    for record in data:
        values = ', '.join([f"'{str(value)}'" for value in record])
        sql_statement = f"""INSERT INTO sensordata 
    (createdAt, mach, temp)
    VALUES ({values});"""
        sql_statements.append(sql_statement)
    return sql_statements

In [47]:
#Creating a list with the days of the year 2024
start_date = datetime.strptime('2024-01-01', '%Y-%m-%d')
end_date = datetime.strptime('2024-12-31', '%Y-%m-%d')

date_list = generate_date_list(start_date, end_date)

In [50]:
#Defining the parameters wich the sensor data will be created
parameters = {
    '1': {'cicleTime': 75, 'cicleDev': 10, 'maq': 'Iny 1'},
    '2': {'cicleTime': 40, 'cicleDev': 5, 'maq': 'Iny 2'},
    '3': {'cicleTime': 90, 'cicleDev': 18, 'maq': 'Iny 3'},
}